In [177]:
import pandas as pd
import numpy as np
import unidecode
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
from textblob import TextBlob
from googletrans import Translator

In [178]:
data = pd.read_csv("Data\\preguntas.csv", sep=',', encoding = "ISO-8859-1")
data.head(3)

,Pregunta,respuesta,Tema,Pers/empresa,Polarity
0,Hola una pregunta en que perjudica el acta de ...,NaN,Renuncia/Despido/Desahucio,persona,0
1,PARA CALCULAR EL DESAHUCIO SE DEBE TOMAR EN CU...,NaN,Renuncia/Despido/Desahucio,persona,0
2,supuestamente Correa emitió el decreto 225 dod...,NaN,Jubilacion Patronal,persona,0


In [179]:
p2 = []
for p in data.Pregunta:
    p2.append(' '.join(p.split()))
data['Pregunta2'] = p2

In [180]:
tokenizer = RegexpTokenizer(r'\w+')
stemmer = SnowballStemmer("spanish")
translator = Translator()

In [181]:
jbtopdict = np.load('Data\\jbtopdict.npy').item()
rentopdict = np.load('Data\\rentopdict.npy').item()
IESStopdict = np.load('Data\\IESStopdict.npy').item()
CONTtopdict = np.load('Data\\CONTtopdict.npy').item()
OStopdict = np.load('Data\\OStopdict.npy').item()
CONStopdict = np.load('Data\\CONStopdict.npy').item()
JStopdic = np.load('Data\\JStopdict.npy').item()

In [182]:
def top_likelihood(t_dict, sentence):
    lik = -1
    tokens = tokenizer.tokenize(unidecode.unidecode(sentence.lower()))
    for w in tokens:
        w = stemmer.stem(w)
        if w in t_dict:
            #print(w)
            if lik == -1:
                if t_dict[w] == 0:
                    lik = -0.1
                else:
                    lik = t_dict[w]
            else:
                if t_dict[w] == 0:
                    lik -= 0.1
                else:
                    lik += t_dict[w]
    return lik

In [183]:
data.Pregunta2[62]

'le saluda Miguel Zamora le acabe de enviar con correo a la Ventas3 favor para que lea mi correo y me conteste gracias'

In [184]:
top_likelihood(jbtopdict , data.Pregunta2[62])

1.327350427350427

In [185]:
stemmer.stem('contactaria')

'contactari'

In [186]:
def sentiment_polarity(sentence):
    translated = translator.translate(sentence)
    blob = TextBlob(translated.text)
    pol = blob.sentiment[0]
    return pol

In [187]:
def percent_greet(sentence):
    tgreet = ['hol', 'buen', 'tard', 'dias', 'noch']
    count = 0
    tokens = tokenizer.tokenize(unidecode.unidecode(sentence.lower()))
    for w in tokens:
        w = stemmer.stem(w)
        if w in tgreet:
            count += 1            
    return count/len(tokens) 

In [188]:
def n_token(sentence):
    tokens = tokenizer.tokenize(unidecode.unidecode(sentence.lower()))       
    return len(tokens) 

In [189]:
data['jplik'] = [top_likelihood(jbtopdict, x) for x in data.Pregunta2]
data['renlik'] = [top_likelihood(rentopdict, x) for x in data.Pregunta2]
data['IESSlik'] = [top_likelihood(IESStopdict, x) for x in data.Pregunta2]
data['CONTlik'] = [top_likelihood(CONTtopdict, x) for x in data.Pregunta2]
data['OSlik'] = [top_likelihood(OStopdict, x) for x in data.Pregunta2]
data['CONSlik'] = [top_likelihood(CONStopdict, x) for x in data.Pregunta2]
data['JSlik'] = [top_likelihood(JStopdic, x) for x in data.Pregunta2]

In [190]:
data['polarity'] = [sentiment_polarity(x) for x in data.Pregunta2]

In [191]:
data['p_greet'] = [percent_greet(x) for x in data.Pregunta2]

In [192]:
data['n_token'] = [n_token(x) for x in data.Pregunta2]

In [193]:
data.head(3)

,Pregunta,respuesta,Tema,Pers/empresa,Polarity,Pregunta2,jplik,renlik,IESSlik,CONTlik,OSlik,CONSlik,JSlik,polarity,p_greet,n_token
0,Hola una pregunta en que perjudica el acta de ...,NaN,Renuncia/Despido/Desahucio,persona,0,Hola una pregunta en que perjudica el acta de ...,-0.057143,2.857143,-0.300000,-0.300000,-0.3,-0.3,-0.300000,0.0,0.076923,13
1,PARA CALCULAR EL DESAHUCIO SE DEBE TOMAR EN CU...,NaN,Renuncia/Despido/Desahucio,persona,0,PARA CALCULAR EL DESAHUCIO SE DEBE TOMAR EN CU...,1.282051,1.439103,0.178846,-0.300000,-0.3,-0.3,-0.300000,0.0,0.000000,17
2,supuestamente Correa emitió el decreto 225 dod...,NaN,Jubilacion Patronal,persona,0,supuestamente Correa emitió el decreto 225 dod...,4.085470,-0.146154,-0.094872,0.044444,-0.5,-0.5,-0.288889,0.0,0.000000,30


In [194]:
data.to_csv("Data\\preguntas2.csv", index=False)